# A Crew to Tailor Job Applications

A multi-agent system that will adjust the provided resume (in markdown format) so it matches the target job description better.

In [4]:
# provide the path and file name for you resume file, and job posting url here

job_posting_url = 'https://job-boards.greenhouse.io/ziprecruiter/jobs/7114545?gh_src=89c6ee071us'
job_title = "Data Scientist"
resume_file = "./Resume.md"

# set up inputs for the crew
job_application_inputs = {
    'job_posting_url': job_posting_url,
    'job_title': job_title
}

logfile = './test_log.txt'


## Setting up the environment

In [ ]:
# python -m venv .venv
# source .crewai_venv/bin/activate
# !pip install crewai crewai_tools langchain_community langchain_openai

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import os
import logging

# Import libraries, APIs and LLM
from crewai import Agent, Task, Crew
from crewai.utilities.paths import db_storage_path
from crewai_tools import ScrapeWebsiteTool, FileReadTool, SerperDevTool, MDXSearchTool

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

logging.basicConfig(filename=logfile, 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s', 
                    filemode='a')
logging.info(f"Storage location: {db_storage_path()}")

# set up crewAI tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume= FileReadTool(resume_file)
mdx_tool = MDXSearchTool(mdx=resume_file)

#mdx_tool.run(search_query="only get work experience")



In [3]:
from utils import LoggedLLM

deepseek_llm = LoggedLLM(
    model="deepseek-chat",
    base_url="https://api.deepseek.com/v1",
    api_key=os.getenv("DEEPSEEK_API_KEY")
)

#response = deepseek_llm.call("Hello")
#print(response)

## Simplified Crew for Testing

In [5]:
chatty = False

# Simplified Agent (reusable)
test_research = Agent(
    role="Test Research Agent",
    goal="Summarize content briefly.",
    tools=[],
    llm=deepseek_llm,
    verbose=chatty,
    backstory="You are a placeholder test research agent."
)
test_profiler = Agent(
    role="Test Profiler Agent",
    goal="Refine and optimize the resume.",
    tools = [read_resume],
    llm=deepseek_llm,
    verbose=chatty,
    backstory="You are another placeholder test profiler agent."
)
# Minimal Tasks
test_research_task = Task(
    description="Summarize job posting from {job_posting_url} in 3 bullet points.",
    expected_output="3 short bullet points about the job posting.",
    agent=test_research
)

test_profile_task = Task(
    description="Summarize resume content in 3 bullet points.",
    expected_output="3 short bullet points about the resume.",
    agent=test_profiler,
    context=[test_research_task]
)

test_resume_strategy_task = Task(
    description="Suggest 2 resume improvements based on job posting + profile.",
    expected_output="2 short suggestions.",
    agent=test_profiler,
    context=[test_research_task, test_profile_task]
)

test_interview_preparation_task = Task(
    description="Generate 2 interview questions based on resume + job posting.",
    expected_output="2 questions only.",
    agent=test_research,
    context=[test_research_task, test_profile_task, test_resume_strategy_task]
)

# Test crew
simplify_crew = Crew(
    agents=[test_research, test_profiler],
    tasks=[test_research_task, test_profile_task, test_resume_strategy_task, test_interview_preparation_task],
    verbose=chatty,
    memory=False,        # turn off memory for test runs
    output_log_file=logfile # keep logs for analysis
)

result = simplify_crew.kickoff(inputs=job_application_inputs)
logging.info(f"[USAGE] My aggregated token usage: {deepseek_llm.usage_totals}")

16:02:46 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
16:02:50 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
16:02:50 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
16:02:56 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
16:02:56 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
16:03:08 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
16:03:08 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
16:03:14 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
16:03:14 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
16:03:28 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
16:03:28 - LiteLLM:INFO: 

## Creating Agents

In [ ]:
chatty = False
# Agent 1: Researcher
researcher = Agent(
    role="Job Researcher for {job_title}",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants. ",
         #"Do not include or echo the raw website content in your answer.",
    tools = [scrape_tool],
    llm=deepseek_llm,
    verbose=chatty,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for {job_title}",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market. ",
         #"Do not include or echo the raw file or website content in your answer.",
    tools = [#scrape_tool, 
             read_resume],
    llm=deepseek_llm,
    verbose=chatty,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for {job_title}",
    goal="Find all the best ways to make a "
         "resume stand out in the job market. ",
         #"Do not include or echo the raw file or website content in your answer.",
    tools = [#scrape_tool,
             read_resume],
    llm=deepseek_llm,
    verbose=chatty,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements. ",
         #"Do not include or echo the raw file or website content in your answer.",
    tools = [read_resume],
    llm=deepseek_llm,
    verbose=chatty,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements. "
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=False
)

# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile. "
        "Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=False
)

# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

# Task for Resume Strategist Agent: Create a Cover Letter to Highlight Matches
cover_letter_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, craft a cover letter to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "letter content, but don't make up any information. "
        "All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An well crafted cover letter that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="cover_letter.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           cover_letter_task,
           interview_preparation_task],

    verbose=chatty,
    memory=False,
    output_log_file=logfile
)

In [ ]:
#result = researcher.kickoff('What are the key requirements for job provided at https://job-boards.greenhouse.io/ziprecruiter/jobs/7114545?gh_src=89c6ee071us')
#result = profiler.kickoff('What can you tell me about this job applicant?')
#result = resume_strategist.kickoff('Refine the job applicant resume')
# result = interview_preparer.kickoff('Prepare 5 interview questions')
# print(result.usage_metrics)
# print(result.raw)

## Running the Crew

In [ ]:
job_posting_url = 'https://job-boards.greenhouse.io/ziprecruiter/jobs/7114545?gh_src=89c6ee071us'
job_title = "Data Scientist"
resume_file = "./Resume.md"

# set up inputs for the crew
job_application_inputs = {
    'job_posting_url': job_posting_url,
    'job_title': job_title
}

In [ ]:
### this execution will take a few minutes to run
logging.info("Running instance for job application strategy")
result = job_application_crew.kickoff(inputs=job_application_inputs)
logging.info(f"[USAGE] My aggregated token usage: {deepseek_llm.usage_totals}")

16:04:04 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
16:04:12 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
16:04:12 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
16:04:32 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
16:04:32 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
16:04:38 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
16:04:38 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
16:05:36 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
16:05:36 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= deepseek-chat; provider = deepseek
16:05:41 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
16:05:41 - LiteLLM:INFO: 

- Dislplay the generated `tailored_resume.md` file.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))